In [1]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np

/Users/akshay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [ ]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        
        optimizer = Adam(0.0002, 0.5)
        
        #Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
                                   optimizer = optimizer,
                                   metrics = ['accuracy'])
        
        #Build the generator
        self.generator = self.build_generator()
        
        #The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        #For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        #The discriminator takes generated 
        #images as input and determines validity
        validity = self.discriminator(img)
        
        #The combined model (stacked generator and discriminator)
        #Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        
    def build_generator(self):
        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1,activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size = 128, sample_interval=50):

        #Load the dataset
        (X_train, _), (_,_) = mnist.load_data()

        #Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        #Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            #######Train discriminator
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0,1,(batch_size, self.latent_dim))

            #generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            #Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            #######Train generator
            noise = np.random.normal(0,1,(batch_size, self.latent_dim))

            #Train gen to have discriminator label samples as valid
            g_loss = self.combined.train_on_batch(noise, valid)

            #plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            #if at save interval, save generated image samples

            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                    
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0,1,(r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        #rescale images 0-1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
            
#         if __name__ == '__main__':
#             gan = GAN()
#             gan.train(epochs=30000, batch_size=32, sample_interval=200)

In [ ]:
gan = GAN()

In [ ]:
gan.train(epochs=5000, batch_size=32, sample_interval=200)

In [40]:
noise = np.random.normal(0,1,(128, 100))

In [43]:
noise

array([[-0.13461111,  1.51431114, -0.01926354, ..., -0.00964456,
        -0.50521095, -0.02747864],
       [-0.08401962, -0.27350335,  1.76438736, ...,  0.60855884,
         0.64127412, -0.19113987],
       [ 0.17852331,  1.19377867, -0.85452346, ...,  1.28840741,
         0.81314043, -0.51632233],
       ...,
       [-2.34801392, -0.18149789, -1.54206513, ..., -0.58451825,
         0.98382412,  0.36513076],
       [-1.583219  ,  0.22002545,  1.04373214, ...,  0.98498183,
        -1.18830867,  0.41660953],
       [ 0.90064248, -0.01551946, -0.01406643, ..., -0.03048208,
         1.01373637,  2.17033617]])

In [12]:
np.expand_dims?

In [56]:
inn = Input(shape=(4,5))
out = inn.sum()

In [57]:
m = Model(inn, out)

ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: Sum{acc_dtype=float64}.0

In [51]:
m.

<bound method Model.compile of <keras.engine.training.Model object at 0x1c1b64b550>>

In [38]:
np.random.randint(0,100,3)

array([ 6, 13, 36])

# Breakdown

In [6]:
def build_generator(latent_dim, img_shape):
    model = Sequential()

    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

In [7]:
build_generator(100, (28,28,1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
__________

In [8]:
def build_discriminator(img_shape):
    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1,activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [9]:
build_discriminator((28,28,1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [22]:
X_train[1].shape

(28, 28)

In [ ]:
plt.imshow(X_train[1], cmap='gray')

In [31]:
xxx = np.expand_dims(X_train[1], axis=2)